## Imports

In [1]:
#  IMPORTING LIBRARIES
import datetime as dt
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

#  Applying settings to plotly.offline for visualization
init_notebook_mode(connected=True)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics

import sklearn
from sklearn import preprocessing

from statsmodels.tools.eval_measures import rmse
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

from keras.preprocessing.sequence import TimeseriesGenerator

Using TensorFlow backend.


## Prediction Models
### Linear Regression Model

### LSTM Model

## Decision Models